In [4]:
import os
import json
import clang.cindex
import clang.enumerations
import csv
import numpy as np
import re 
import warnings
import argparse
import sys
from tqdm import tqdm

In [5]:
type1 = "callee"
type2 = "d"
type1 == type2 if type1 == "callee" else True

False

In [6]:
warnings.filterwarnings('ignore')
# KeyPoint로 지정 될 함수들
l_funcs = ['StrNCat', 'getaddrinfo', '_ui64toa', 'fclose', 'pthread_mutex_lock', 'gets_s', 'sleep', 
           '_ui64tot', 'freopen_s', '_ui64tow', 'send', 'lstrcat', 'HMAC_Update', '__fxstat', 'StrCatBuff', 
           '_mbscat', '_mbstok_s', '_cprintf_s', 'ldap_search_init_page', 'memmove_s', 'ctime_s', 'vswprintf', 
           'vswprintf_s', '_snwprintf', '_gmtime_s', '_tccpy', '*RC6*', '_mbslwr_s', 'random', 
           '__wcstof_internal', '_wcslwr_s', '_ctime32_s', 'wcsncat*', 'MD5_Init', '_ultoa', 
           'snprintf', 'memset', 'syslog', '_vsnprintf_s', 'HeapAlloc', 'pthread_mutex_destroy', 
           'ChangeWindowMessageFilter', '_ultot', 'crypt_r', '_strupr_s_l', 'LoadLibraryExA', '_strerror_s', 
           'LoadLibraryExW', 'wvsprintf', 'MoveFileEx', '_strdate_s', 'SHA1', 'sprintfW', 'StrCatNW', 
           '_scanf_s_l', 'pthread_attr_init', '_wtmpnam_s', 'snscanf', '_sprintf_s_l', 'dlopen', 
           'sprintfA', 'timed_mutex', 'OemToCharA', 'ldap_delete_ext', 'sethostid', 'popen', 'OemToCharW', 
           '_gettws', 'vfork', '_wcsnset_s_l', 'sendmsg', '_mbsncat', 'wvnsprintfA', 'HeapFree', '_wcserror_s', 
           'realloc', '_snprintf*', 'wcstok', '_strncat*', 'StrNCpy', '_wasctime_s', 'push*', '_lfind_s', 
           'CC_SHA512', 'ldap_compare_ext_s', 'wcscat_s', 'strdup', '_chsize_s', 'sprintf_s', 'CC_MD4_Init', 
           'wcsncpy', '_wfreopen_s', '_wcsupr_s', '_searchenv_s', 'ldap_modify_ext_s', '_wsplitpath', 
           'CC_SHA384_Final', 'MD2', 'RtlCopyMemory', 'lstrcatW', 'MD4', 'MD5', '_wcstok_s_l', '_vsnwprintf_s', 
           'ldap_modify_s', 'strerror', '_lsearch_s', '_mbsnbcat_s', '_wsplitpath_s', 'MD4_Update', '_mbccpy_s', 
           '_strncpy_s_l', '_snprintf_s', 'CC_SHA512_Init', 'fwscanf_s', '_snwprintf_s', 'CC_SHA1', 'swprintf', 
           'fprintf', 'EVP_DigestInit_ex', 'strlen', 'SHA1_Init', 'strncat', '_getws_s', 'CC_MD4_Final', 
           'wnsprintfW', 'lcong48', 'lrand48', 'write', 'HMAC_Init', '_wfopen_s', 'wmemchr', '_tmakepath', 
           'wnsprintfA', 'lstrcpynW', 'scanf_s', '_mbsncpy_s_l', '_localtime64_s', 'fstream.open', '_wmakepath', 
           'Connection.open', '_tccat', 'valloc', 'setgroups', 'unlink', 'fstream.put', 'wsprintfA', '*SHA1*', 
           '_wsearchenv_s', 'ualstrcpyA', 'CC_MD5_Update', 'strerror_s', 'HeapCreate', 'ualstrcpyW', '__xstat', 
           '_wmktemp_s', 'StrCatChainW', 'ldap_search_st', '_mbstowcs_s_l', 'ldap_modify_ext', '_mbsset_s', 
           'strncpy_s', 'move', 'execle', 'StrCat', 'xrealloc', 'wcsncpy_s', '_tcsncpy*', 'execlp', 
           'RIPEMD160_Final', 'ldap_search_s', 'EnterCriticalSection', '_wctomb_s_l', 'fwrite', '_gmtime64_s', 
           'sscanf_s', 'wcscat', '_strupr_s', 'wcrtomb_s', 'VirtualLock', 'ldap_add_ext_s', '_mbscpy', 
           '_localtime32_s', 'lstrcpy', '_wcsncpy*', 'CC_SHA1_Init', '_getts', '_wfopen', '__xstat64', 
           'strcoll', '_fwscanf_s_l', '_mbslwr_s_l', 'RegOpenKey', 'makepath', 'seed48', 'CC_SHA256', 
           'sendto', 'execv', 'CalculateDigest', 'memchr', '_mbscpy_s', '_strtime_s', 'ldap_search_ext_s', 
           '_chmod', 'flock', '__fxstat64', '_vsntprintf', 'CC_SHA256_Init', '_itoa_s', '__wcserror_s', 
           '_gcvt_s', 'fstream.write', 'sprintf', 'recursive_mutex', 'strrchr', 'gethostbyaddr', '_wcsupr_s_l', 
           'strcspn', 'MD5_Final', 'asprintf', '_wcstombs_s_l', '_tcstok', 'free', 'MD2_Final', 'asctime_s', 
           '_alloca', '_wputenv_s', '_wcsset_s', '_wcslwr_s_l', 'SHA1_Update', 'filebuf.sputc', 'filebuf.sputn', 
           'SQLConnect', 'ldap_compare', 'mbstowcs_s', 'HMAC_Final', 'pthread_condattr_init', '_ultow_s', 'rand', 
           'ofstream.put', 'CC_SHA224_Final', 'lstrcpynA', 'bcopy', 'system', 'CreateFile*', 'wcscpy_s', 
           '_mbsnbcpy*', 'open', '_vsnwprintf', 'strncpy', 'getopt_long', 'CC_SHA512_Final', '_vsprintf_s_l', 
           'scanf', 'mkdir', '_localtime_s', '_snprintf', '_mbccpy_s_l', 'memcmp', 'final', '_ultoa_s', 
           'lstrcpyW', 'LoadModule', '_swprintf_s_l', 'MD5_Update', '_mbsnset_s_l', '_wstrtime_s', '_strnset_s', 
           'lstrcpyA', '_mbsnbcpy_s', 'mlock', 'IsBadHugeWritePtr', 'copy', '_mbsnbcpy_s_l', 'wnsprintf', 
           'wcscpy', 'ShellExecute', 'CC_MD4', '_ultow', '_vsnwprintf_s_l', 'lstrcpyn', 'CC_SHA1_Final', 
           'vsnprintf', '_mbsnbset_s', '_i64tow', 'SHA256_Init', 'wvnsprintf', 'RegCreateKey', 'strtok_s', 
           '_wctime32_s', '_i64toa', 'CC_MD5_Final', 'wmemcpy', 'WinExec', 'CreateDirectory*', 
           'CC_SHA256_Update', '_vsnprintf_s_l', 'jrand48', 'wsprintf', 'ldap_rename_ext_s', 'filebuf.open', 
           '_wsystem', 'SHA256_Update', '_cwscanf_s', 'wsprintfW', '_sntscanf', '_splitpath', 'fscanf_s', 
           'strpbrk', 'wcstombs_s', 'wscanf', '_mbsnbcat_s_l', 'strcpynA', 'pthread_cond_init', 'wcsrtombs_s', 
           '_wsopen_s', 'CharToOemBuffA', 'RIPEMD160_Update', '_tscanf', 'HMAC', 'StrCCpy', 'Connection.connect', 
           'lstrcatn', '_mbstok', '_mbsncpy', 'CC_SHA384_Update', 'create_directories', 'pthread_mutex_unlock', 
           'CFile.Open', 'connect', '_vswprintf_s_l', '_snscanf_s_l', 'fputc', '_wscanf_s', '_snprintf_s_l', 
           'strtok', '_strtok_s_l', 'lstrcatA', 'snwscanf', 'pthread_mutex_init', 'fputs', 'CC_SHA384_Init', 
           '_putenv_s', 'CharToOemBuffW', 'pthread_mutex_trylock', '__wcstoul_internal', '_memccpy', 
           '_snwprintf_s_l', '_strncpy*', 'wmemset', 'MD4_Init', '*RC4*', 'strcpyW', '_ecvt_s', 'memcpy_s', 
           'erand48', 'IsBadHugeReadPtr', 'strcpyA', 'HeapReAlloc', 'memcpy', 'ldap_rename_ext', 'fopen_s', 
           'srandom', '_cgetws_s', '_makepath', 'SHA256_Final', 'remove', '_mbsupr_s', 'pthread_mutexattr_init', 
           '__wcstold_internal', 'StrCpy', 'ldap_delete', 'wmemmove_s', '_mkdir', 'strcat', '_cscanf_s_l', 
           'StrCAdd', 'swprintf_s', '_strnset_s_l', 'close', 'ldap_delete_ext_s', 'ldap_modrdn', 'strchr', 
           '_gmtime32_s', '_ftcscat', 'lstrcatnA', '_tcsncat', 'OemToChar', 'mutex', 'CharToOem', 'strcpy_s', 
           'lstrcatnW', '_wscanf_s_l', '__lxstat64', 'memalign', 'MD2_Init', 'StrCatBuffW', 'StrCpyN', 'CC_MD5', 
           'StrCpyA', 'StrCatBuffA', 'StrCpyW', 'tmpnam_r', '_vsnprintf', 'strcatA', 'StrCpyNW', '_mbsnbset_s_l', 
           'EVP_DigestInit', '_stscanf', 'CC_MD2', '_tcscat', 'StrCpyNA', 'xmalloc', '_tcslen', '*MD4*', 
           'vasprintf', 'strxfrm', 'chmod', 'ldap_add_ext', 'alloca', '_snscanf_s', 'IsBadWritePtr', 'swscanf_s', 
           'wmemcpy_s', '_itoa', '_ui64toa_s', 'EVP_DigestUpdate', '__wcstol_internal', '_itow', 'StrNCatW', 
           'strncat_s', 'ualstrcpy', 'execvp', '_mbccat', 'EVP_MD_CTX_init', 'assert', 'ofstream.write', 
           'ldap_add', '_sscanf_s_l', 'drand48', 'CharToOemW', 'swscanf', '_itow_s', 'RIPEMD160_Init', 
           'CopyMemory', 'initstate', 'getpwuid', 'vsprintf', '_fcvt_s', 'CharToOemA', 'setuid', 'malloc', 
           'StrCatNA', 'strcat_s', 'srand', 'getwd', '_controlfp_s', 'olestrcpy', '__wcstod_internal', 
           '_mbsnbcat', 'lstrncat', 'des_*', 'CC_SHA224_Init', 'set*', 'vsprintf_s', 'SHA1_Final', '_umask_s', 
           'gets', 'setstate', 'wvsprintfW', 'LoadLibraryEx', 'ofstream.open', 'calloc', '_mbstrlen', 
           '_cgets_s', '_sopen_s', 'IsBadStringPtr', 'wcsncat_s', 'add*', 'nrand48', 'create_directory', 
           'ldap_search_ext', '_i64toa_s', '_ltoa_s', '_cwscanf_s_l', 'wmemcmp', '__lxstat', 'lstrlen', 
           'pthread_condattr_destroy', '_ftcscpy', 'wcstok_s', '__xmknod', 'pthread_attr_destroy', 'sethostname', 
           '_fscanf_s_l', 'StrCatN', 'RegEnumKey', '_tcsncpy', 'strcatW', 'AfxLoadLibrary', 'setenv', 'tmpnam', 
           '_mbsncat_s_l', '_wstrdate_s', '_wctime64_s', '_i64tow_s', 'CC_MD4_Update', 'ldap_add_s', '_umask', 
           'CC_SHA1_Update', '_wcsset_s_l', '_mbsupr_s_l', 'strstr', '_tsplitpath', 'memmove', '_tcscpy', 
           'vsnprintf_s', 'strcmp', 'wvnsprintfW', 'tmpfile', 'ldap_modify', '_mbsncat*', 'mrand48', 'sizeof', 
           'StrCatA', '_ltow_s', '*desencrypt*', 'StrCatW', '_mbccpy', 'CC_MD2_Init', 'RIPEMD160', 'ldap_search', 
           'CC_SHA224', 'mbsrtowcs_s', 'update', 'ldap_delete_s', 'getnameinfo', '*RC5*', '_wcsncat_s_l', 
           'DriverManager.getConnection', 'socket', '_cscanf_s', 'ldap_modrdn_s', '_wopen', 'CC_SHA256_Final', 
           '_snwprintf*', 'MD2_Update', 'strcpy', '_strncat_s_l', 'CC_MD5_Init', 'mbscpy', 'wmemmove', 
           'LoadLibraryW', '_mbslen', '*alloc', '_mbsncat_s', 'LoadLibraryA', 'fopen', 'StrLen', 'delete', 
           '_splitpath_s', 'CreateFileTransacted*', 'MD4_Final', '_open', 'CC_SHA384', 'wcslen', 'wcsncat', 
           '_mktemp_s', 'pthread_mutexattr_destroy', '_snwscanf_s', '_strset_s', '_wcsncpy_s_l', 'CC_MD2_Final', 
           '_mbstok_s_l', 'wctomb_s', 'MySQL_Driver.connect', '_snwscanf_s_l', '*_des_*', 'LoadLibrary', 
           '_swscanf_s_l', 'ldap_compare_s', 'ldap_compare_ext', '_strlwr_s', 'GetEnvironmentVariable', 
           'cuserid', '_mbscat_s', 'strspn', '_mbsncpy_s', 'ldap_modrdn2', 'LeaveCriticalSection', 'CopyFile', 
           'getpwd', 'sscanf', 'creat', 'RegSetValue', 'ldap_modrdn2_s', 'CFile.Close', '*SHA_1*', 
           'pthread_cond_destroy', 'CC_SHA512_Update', '*RC2*', 'StrNCatA', '_mbsnbcpy', '_mbsnset_s', 
           'crypt', 'excel', '_vstprintf', 'xstrdup', 'wvsprintfA', 'getopt', 'mkstemp', '_wcsnset_s', 
           '_stprintf', '_sntprintf', 'tmpfile_s', 'OpenDocumentFile', '_mbsset_s_l', '_strset_s_l', 
           '_strlwr_s_l', 'ifstream.open', 'xcalloc', 'StrNCpyA', '_wctime_s', 'CC_SHA224_Update', '_ctime64_s', 
           'MoveFile', 'chown', 'StrNCpyW', 'IsBadReadPtr', '_ui64tow_s', 'IsBadCodePtr', 'getc', 
           'OracleCommand.ExecuteOracleScalar', 'AccessDataSource.Insert', 'IDbDataAdapter.FillSchema', 
           'IDbDataAdapter.Update', 'GetWindowText*', 'SendMessage', 'SqlCommand.ExecuteNonQuery', 'streambuf.sgetc', 
           'streambuf.sgetn', 'OracleCommand.ExecuteScalar', 'SqlDataSource.Update', '_Read_s', 'IDataAdapter.Fill', 
           '_wgetenv', '_RecordsetPtr.Open*', 'AccessDataSource.Delete', 'Recordset.Open*', 'filebuf.sbumpc', 'DDX_*', 
           'RegGetValue', 'fstream.read*', 'SqlCeCommand.ExecuteResultSet', 'SqlCommand.ExecuteXmlReader', 'main', 
           'streambuf.sputbackc', 'read', 'm_lpCmdLine', 'CRichEditCtrl.Get*', 'istream.putback', 
           'SqlCeCommand.ExecuteXmlReader', 'SqlCeCommand.BeginExecuteXmlReader', 'filebuf.sgetn', 
           'OdbcDataAdapter.Update', 'filebuf.sgetc', 'SQLPutData', 'recvfrom', 'OleDbDataAdapter.FillSchema', 
           'IDataAdapter.FillSchema', 'CRichEditCtrl.GetLine', 'DbDataAdapter.Update', 'SqlCommand.ExecuteReader', 
           'istream.get', 'ReceiveFrom', '_main', 'fgetc', 'DbDataAdapter.FillSchema', 'kbhit', 'UpdateCommand.Execute*', 
           'Statement.execute', 'fgets', 'SelectCommand.Execute*', 'getch', 'OdbcCommand.ExecuteNonQuery', 
           'CDaoQueryDef.Execute', 'fstream.getline', 'ifstream.getline', 'SqlDataAdapter.FillSchema', 
           'OleDbCommand.ExecuteReader', 'Statement.execute*', 'SqlCeCommand.BeginExecuteNonQuery', 
           'OdbcCommand.ExecuteScalar', 'SqlCeDataAdapter.Update', 'sendmessage', 'mysqlpp.DBDriver', 'fstream.peek', 
           'Receive', 'CDaoRecordset.Open', 'OdbcDataAdapter.FillSchema', '_wgetenv_s', 'OleDbDataAdapter.Update', 
           'readsome', 'SqlCommand.BeginExecuteXmlReader', 'recv', 'ifstream.peek', '_Main', '_tmain', '_Readsome_s', 
           'SqlCeCommand.ExecuteReader', 'OleDbCommand.ExecuteNonQuery', 'fstream.get', 'IDbCommand.ExecuteScalar', 
           'filebuf.sputbackc', 'IDataAdapter.Update', 'streambuf.sbumpc', 'InsertCommand.Execute*', 'RegQueryValue', 
           'IDbCommand.ExecuteReader', 'SqlPipe.ExecuteAndSend', 'Connection.Execute*', 'getdlgtext', 'ReceiveFromEx', 
           'SqlDataAdapter.Update', 'RegQueryValueEx', 'SQLExecute', 'pread', 'SqlCommand.BeginExecuteReader', 'AfxWinMain', 
           'getchar', 'istream.getline', 'SqlCeDataAdapter.Fill', 'OleDbDataReader.ExecuteReader', 'SqlDataSource.Insert', 
           'istream.peek', 'SendMessageCallback', 'ifstream.read*', 'SqlDataSource.Select', 'SqlCommand.ExecuteScalar', 
           'SqlDataAdapter.Fill', 'SqlCommand.BeginExecuteNonQuery', 'getche', 'SqlCeCommand.BeginExecuteReader', 'getenv', 
           'streambuf.snextc', 'Command.Execute*', '_CommandPtr.Execute*', 'SendNotifyMessage', 'OdbcDataAdapter.Fill', 
           'AccessDataSource.Update', 'fscanf', 'QSqlQuery.execBatch', 'DbDataAdapter.Fill', 'cin', 
           'DeleteCommand.Execute*', 'QSqlQuery.exec', 'PostMessage', 'ifstream.get', 'filebuf.snextc', 
           'IDbCommand.ExecuteNonQuery', 'Winmain', 'fread', 'getpass', 'GetDlgItemTextCCheckListBox.GetCheck', 
           'DISP_PROPERTY_EX', 'pread64', 'Socket.Receive*', 'SACommand.Execute*', 'SQLExecDirect', 
           'SqlCeDataAdapter.FillSchema', 'DISP_FUNCTION', 'OracleCommand.ExecuteNonQuery', 'CEdit.GetLine', 
           'OdbcCommand.ExecuteReader', 'CEdit.Get*', 'AccessDataSource.Select', 'OracleCommand.ExecuteReader', 
           'OCIStmtExecute', 'getenv_s', 'DB2Command.Execute*', 'OracleDataAdapter.FillSchema', 'OracleDataAdapter.Fill', 
           'CComboBox.Get*', 'SqlCeCommand.ExecuteNonQuery', 'OracleCommand.ExecuteOracleNonQuery', 'mysqlpp.Query', 
           'istream.read*', 'CListBox.GetText', 'SqlCeCommand.ExecuteScalar', 'ifstream.putback', 'readlink', 
           'CHtmlEditCtrl.GetDHtmlDocument', 'PostThreadMessage', 'CListCtrl.GetItemText', 'OracleDataAdapter.Update', 
           'OleDbCommand.ExecuteScalar', 'stdin', 'SqlDataSource.Delete', 'OleDbDataAdapter.Fill', 'fstream.putback', 
           'IDbDataAdapter.Fill', '_wspawnl', 'fwprintf', 'sem_wait', '_unlink', 'ldap_search_ext_sW', 'signal', 'PQclear', 
           'PQfinish', 'PQexec', 'PQresultStatus', 'atoi'
           ]
# key의 number 추출           
def extract_key_number(idx, nodes):
    while idx >= 0:
        c_node = nodes[idx]
        if 'key' in c_node.keys():
            key = c_node['key']
            if key.strip() != '': # 공백 제거
                try:
                    ln = int(key)
                    return ln
                except:
                    pass
        idx -= 1
    return -1

# foward slice 생성    
def create_forward_slice(adjacency_list, line_no):
    sliced_lines = set()    # 슬라이스 된 라인 번호들을 저장할 집합    
    sliced_lines.add(line_no)   # create_adjacency_slices에서 no를 통해 받은 라인별로 슬라이스 라인 집합에 추가
    # DFS를 위한 스택 선언
    stack = list()
    stack.append(line_no)
    
    # 스택의 요소가 0이 되기 전까지 반복
    while len(stack) != 0: 
        
        # 현재의 라인 번호 빼오기
        cur = stack.pop()
        # 해당 라인이 슬라이스된 라인 집합에 없으면
        if cur not in sliced_lines: 
            sliced_lines.add(cur)   # cur에 해당하는 데이터를 sliced_line에 추가
            
        # 현재 라인 번호에 연결된 인접 노드를 가져온다(while)  
        adjacents = adjacency_list[cur] 
        
        # 인접 노드들 중 아직 슬라이스 되지 않은 노드를 스택에 추가
        for node in adjacents:
            if node not in sliced_lines:
                stack.append(node)
    sliced_lines = sorted(sliced_lines) # 슬라이스된 라인 번호 정렬
    return sliced_lines

# control과 data adjacention?
def combine_control_and_data_adjacents(adjacency_list):
    cgraph = {}
    for ln in adjacency_list:
        cgraph[ln] = set()
        cgraph[ln] = cgraph[ln].union(adjacency_list[ln][0]) #합집합 구하기
        cgraph[ln] = cgraph[ln].union(adjacency_list[ln][1])
    return cgraph #최종적으론 combined graph 반환

# adjacency_list
#현재 예상은, 위의 foward를 뒤집기 위해 사용하는 것으로 예상 중
def invert_graph(adjacency_list):
    igraph = {} # 역순으로 만들 인접 리스트
    for ln in adjacency_list.keys():    #키 값이 아마 line number로 오는 것 같음
        igraph[ln] = set()
    for ln in adjacency_list:
        adj = adjacency_list[ln]
        for node in adj:
            igraph[node].add(ln)
    return igraph
    pass

## adjacency_data_to_id 함수에 사용 ##  
def invert_graph_data(adjacency_list):
    igraph = {}
    for ln in adjacency_list:
        adj = adjacency_list[ln]
        for node in adj:
            if node not in igraph:
                igraph[node] = set()
            igraph[node].add(ln)
    return igraph
    pass

## backward slice를 만드는 함수를 만든게 아닌, foward를 뒤집는 함수를 만들어서 생성
def create_backward_slice(adjacency_list, line_no):
    inverted_adjacency_list = invert_graph(adjacency_list)
    return create_forward_slice(inverted_adjacency_list, line_no)

# node, edge등을 만들기 위해 csv파일을 읽음
def read_csv(csv_file_path):
    data = []
    with open(csv_file_path) as fp:
        header = fp.readline()
        header = header.strip()
        h_parts = [hp.strip() for hp in header.split('\t')]
        for line in fp:
            line = line.strip()
            instance = {}
            lparts = line.split('\t')
            for i, hp in enumerate(h_parts):
                if i < len(lparts):
                    content = lparts[i].strip()
                else:
                    content = ''
                instance[hp] = content
            data.append(instance)
        return data

## 노드 ID를 기반으로 인접 리스트 생성 ##
def create_adjacency_list_nodeID(line_numbers, node_id_to_line_numbers, edges, target_edge=['CONTROLS','REACHES']):
    adjacency_list = {} 
    set_list = []   # 시작 노드를 추적하기 위한 리스트
    target_edge_length = len(target_edge)   # target_edge의 갯수
    
    # 인접리스트 초기화
    for ln in set(line_numbers):
        adjacency_list[ln] = []
        for i in range(target_edge_length):
            adjacency_list[ln].append(set())
    
    # 엣지 리스트 순회, 인접 리스트 업데이트
    for edge in edges:
        #엣의 타입을 추출
        edge_type = edge['type'].strip()    
        start_node_id = edge['start'].strip() # 시작 노드 아이디 추출
        end_node_id = edge['end'].strip()   # 끝 노드 아이디 추출
        
        #엣지 타입이 target_edge 유형에 포함되는지 확인
        if edge_type in target_edge:
            #시작 노드 ID가 처리 된 적이 없는 경우  시작 노드 ID에 대한 리스트를 초기화한다
            if start_node_id not in set_list: 
                adjacency_list[start_node_id] = []
                
                # target_edge의 갯수 만큼 각 엣지유형에 대한 빈 집합 추가
                for i in range(target_edge_length):
                    adjacency_list[start_node_id].append(set())
                set_list.append(start_node_id) # 시작 노드 ID를 처리된 리스트에 추가
                # edge 유형별로 끝 노드 ID를 시작 노드 집합에 추가
            adjacency_list[start_node_id][target_edge.index(edge_type)].add(end_node_id)   
    return adjacency_list # 인접리스트 반환

# 노드의 location, type, linenumber 등을 추출
def extract_nodes_with_location_info(nodes):
    node_indices = {}   # 각 노드 ID와 해당 노드의 인덱스 매핑을 위한 dict
    node_ids = []   # node_indices의 키가 될 node id 리스트
    line_numbers = [] #
    node_id_to_line_number = {} # 노드ID -> ln으로 변경
    node_function_to_id = {} # 함수 이름과 해당 함수의 노드 ID 매핑
    node_id_to_function_id = {} # 각 노드 ID와 관련된 함수 ID 매핑
    function_id_to_entry = {}
    function_info = {}
    functions = []
    parameter = 1
    return_statement = 1
    function_id_to_callee = {}
    
    for node_index, node in enumerate(nodes):  
        assert isinstance(node, dict)   # 노드 조건은 딕셔너리
        # function ID와 key(노드 아이디) 값을 추출
        function_id = node['functionId'].strip()
        node_id = node['key'].strip()
        node_ids.append(node_id)
        node_indices[node_id] = node_index
        
        # 노드의 타입이 Function인 경우 함수 이름과 ID를 저장
        if node['type'].strip() == "Function":
            node_function_to_id[node['code'].strip()] = node_id
            function_info['Key'] = node_id
            function_info['FunctionName'] = node['code'].strip()
        
        # function_id가 유효한 경우
        if function_id != "":
            # 노드 타입이 Callee인 경우 호출된 함수 정보 저장
            if node['type'].strip() == "Callee":
                # function_id가 function_id_to_callee에 속한 경우 -> 이미 해당함수 ID가 있는 경우
                if function_id in function_id_to_callee:
                    # 추가된 함수로 저장
                    function_id_to_callee[function_id][node['code'].strip()] = node['key'].strip()
                    
                    # 처음 해당 함수의 ID가 나타나는 경우, 딕셔너리 초기화 후 저장
                else:
                    function_id_to_callee[function_id] = {}
                    function_id_to_callee[function_id][node['code'].strip()] = node['key'].strip()
            node_id_to_function_id[node_id] =  function_id # 노드 ID - 함수 ID 매핑
            
            # 함수 엔트리 포인트인 경우
            if node['code'].strip() == "ENTRY":
                function_id_to_entry[function_id] = node_id # 함수 ID, 엔트리 노드 ID 매핑  
                functions.append(function_info) # 수집된 함수 정보에 추가
                parameter=1
                function_info = {}  # 새로운 함수 정보를 저장하기 위해 초기화
            
            # RetrunStatement 타입인 경우 반환문 정보 저장    
            elif node['type'].strip() =='ReturnStatement':
                function_info['ReturnStatement'+str(return_statement)] = node['code'].strip() # 코드
                function_info['ReturnStatementType'+str(return_statement)] = "return;"  # 타입
            # Parameter 타입인 경우 매개변수 이름 저장
            elif node['type'].strip() == 'Parameter': 
                function_info['Parameter'+str(parameter)] = node['code'].strip()
            # ParameterType 타입인 경우 매개변수 타입 저장    
            elif node['type'].strip() == 'ParameterType':
                function_info['ParameterType'+str(parameter)] = node['code'].strip()
                parameter+=1
        
        # 노드의 위치 정보가 있는 경우
        if 'location' in node.keys():
            location = node['location'] # 위치 정보 추출
            if location == '':
                continue
            line_num = int(location.split(':')[0])  # 위치 정보를 통해 라인넘버 추출
            line_numbers.append(line_num)   #라인번호 리스트 추가
            if 'CompoundStatement' != node['type'].strip():
                node_id_to_line_number[node_id] = line_num
    return function_id_to_callee, functions, function_id_to_entry, node_id_to_function_id,node_function_to_id,node_indices, node_ids,line_numbers,node_id_to_line_number
    pass

## adjacency_data dict의 key 값을 set로 채우는 과정 ##
def create_adjacency_data(line_numbers, edges):
    adjacency_data = {}
    
    # 기본 인접 데이터 초기화
    for ln in set(line_numbers):
        adjacency_data[ln]= set()   # line number가 인덱스
        
    # edge의 리스트를 순회하며, 인접 데이터 업데이트    
    for edge in edges:  
        edge_type = edge['type'].strip()
        
        #엣지의 타입이 REACHES인 경우(Data dependency)
        if edge_type in ['REACHES']:
            edge_var = edge['var'].strip() # 엣지의 var type 데이터 추출
            
            #이때 var에 아무것도 없으면 스킵
            if edge_var == "":
                continue
            
            # 시작 노드, 끝 노드 ID 추출
            start_node_id = edge['start'].strip()
            end_node_id = edge['end'].strip()
            #시작, 끝 노드의 id를 통해 인접데이터의 var 데이터 추가(연관성)
            adjacency_data[start_node_id].add(edge_var)
            adjacency_data[end_node_id].add(edge_var)
    return adjacency_data   #근데 지금 adjacency_data가 모든 라인이 다 잡히는데 왜이럴까

## adj 슬라이스 만들기, foward, backward 합침 ##
def create_adjacency_slices(adjacency_list, line_nos):
    line_nos = set(line_nos)
    all_slice_set = set()
    # 라임 넘버별로 인접 슬라이스를 생성하는 반복문
    for no in line_nos:
        forward_sliced_lines = create_forward_slice(adjacency_list,no)
        backward_sliced_lines = create_backward_slice(adjacency_list, no)
        all_slice_set = all_slice_set.union(set(forward_sliced_lines))
        all_slice_set = all_slice_set.union(set(backward_sliced_lines))
    line_nos = all_slice_set.union(set(all_slice_set))
    return line_nos

# callee function 탐색
def find_function_info(function_info,callee):
    for function in function_info:
        if function['Key'] == callee: 
            return function
    return -1

# ln 추출
def extract_line_number(idx, nodes):
    while idx >= 0:
        c_node = nodes[idx]
        if 'location' in c_node.keys():
            location = c_node['location']
            if location.strip() != '':
                try:
                    ln = int(location.split(':')[0])
                    return ln
                except:
                    pass
        idx -= 1
    return -1

def concat_lines(func,inter_funcs):
    for inter_func in inter_funcs.keys():
        target = inter_func
        insert_data = inter_funcs[inter_func]
        insert_data.reverse()
        i = -1
        for idx ,ln in enumerate(func):
            if ln == target:
                i = idx + 1
                break
        for j in insert_data:
            func.insert(i,j)
    return func
    
def get_CWE(filename):
    CWEID = filename.split("_")[0]
    return re.sub(r'[^0-9]', '',CWEID)

catrgory_dict = {
    "FC" :["CallExpression"],
    "PU" : ["PtrMemberAccess"],
    "AU" : ["ArrayIndexing"],
    "AE" :  ["AdditiveExpression", "MultiplicativeExpression"]
}

## criterion 함수의 리스트를 만드는 함수 -> Keypoint가 될 Statement 찾는 함수 ##
def get_criterion(category, node_ln_to_id, nodes, genTest=False):
    assert category in ["FC","PU","AU","AE"]
    criterion_list = []
    
    if len(nodes) == 0: return []
    key = -1
    # 리스트 순회
    for node_idx, node in enumerate(nodes):
        node_type = node['type'].strip()
        
        # Function 타입인 경우 인덱스를 하나 증가시켜 code를 가져옴
        if node_type == "Function" and not genTest:
            function_name = nodes[node_idx + 1]['code'] 
            print(function_name)
            # 이때 함수명에 아무것도 안들어가면 스킵
            if function_name  is None or function_name.strip() == '':
                    continue
            # 함수명에, bad, good이라는 문자열이 포함된 경우
            if function_name.find("bad") >= 0 or function_name.find("good") >= 0:
                 # line, key 추출 
                 line_no = extract_line_number(node_idx, nodes)
                 key = extract_key_number(node_idx,nodes)
                 
        # 노드타입이 Function은 아니지만, category(FC, PU, AU, AE)에 속한 경우         
        elif node_type in catrgory_dict[category]:
            # CallExpresion, Sizeof 타입인 경우
            if node_type in ["CallExpression", "Sizeof"]:
                
                # 이또한 인덱스 하나 증가, code를 가져온다
                function_name = nodes[node_idx + 1]['code']
                
                # 함수가 없거나, l_funcs(취약점 발생 함수)가 아닌경우 스킵
                if function_name  is None or function_name.strip() == '':
                    continue
                if function_name.strip() not in l_funcs:
                    continue
            # 현재 노드의 코드를 가져온다
            else :
                function_name = nodes[node_idx]['code']
            # 라인번호를 추출, 함수 id를 정수화 한다
            line_no = extract_line_number(node_idx, nodes)
            nfunctionId=int(node['functionId'])
            # key가 nfunctionID와 같으면서 line_no가 보다 큰 경우 해당 리스트에 추가
            if (key == nfunctionId or genTest)and line_no> 0:
                criterion_list.append([node_ln_to_id[line_no],function_name])
    return criterion_list

## 함수 ID를 통해 함수 확인, 특정 조건에 따른 레이블링을 해주는 함수 ##
def calc_label(node_id_to_function, function_id, Sink_flag, Source_flag, genTest):
    label = 0
    if genTest : return -3
    try : 
        functionName = node_id_to_function[function_id]
        # 함수 이름이 main 이면 label = 1
        if functionName.find("main") >=0:
            label = -1
            return label
        f_bad = functionName.find("bad") >= 0
        f_goodG2B = functionName.find("goodG2B") >= 0
        f_Source = functionName.find("Source") >= 0
        f_Sink = functionName.find("Sink") >= 0

        # 첫번째 조건: 함수에 Bad or goodG2B포함 Source, Sink 포함
        if (f_bad and not f_Source and not f_Sink) or (f_goodG2B and not f_Source and not f_Sink):
            # Sink 플래그 true일때 0으로 레이블링
            if Sink_flag:   
                label = 0
            # Source 플래그 true일때 1으로 레이블링    
            elif Source_flag:
                label = 1
            else :
                # G2B가 포함된 경우 0 으로 레이블링
                if f_goodG2B:
                    label = 0
                # Bad가 포함된 경우 0 으로 레이블링
                else:
                    label = 1
        # 두번째 bad, goodG2B 포함하고 Sink가 포함된 경우 1로 레이블링
        elif ( (f_bad or f_goodG2B) and f_Sink ):
            label = 1
        # 기타 경우의 수 0으로 레이플링
        else:
            label = 0
    except:
        label = -2
     
    return label      

In [7]:
# 노드, 엣지 정보 추출 

source_path = r"C:\Users\csw24\Projects\Slicer_2025\data\raw_data\CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01.c"
nodes_file_path =r"C:\Users\csw24\Projects\Slicer_2025\data\cpg\CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01.c\nodes.csv"
edges_file_path =r"C:\Users\csw24\Projects\Slicer_2025\data\cpg\CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01.c\edges.csv"

nc = open(nodes_file_path) 
nodes_file = csv.DictReader(nc, delimiter='\t')
nodes_dict = [node for node in nodes_file]

nodes = read_csv(nodes_file_path)

# 해당 블록에서 사용 안됨
edges = read_csv(edges_file_path)      

file1 = open(source_path,'r')
code = file1.readlines()
file1.close()

# 해당 함수를 통해 불러온 nodes.csv의 정보를 매핑
#
function_id_to_callee, function_info, function_id_to_entry, node_id_to_function_id, node_function_to_id, node_indices, node_ids, line_numbers, node_id_to_ln = extract_nodes_with_location_info(nodes)


line_numbers

[24,
 33,
 34,
 3,
 36,
 37,
 4,
 37,
 38,
 40,
 42,
 43,
 44,
 45,
 46,
 54,
 55,
 3,
 57,
 58,
 4,
 58,
 59,
 61,
 63,
 64,
 65,
 66,
 67,
 71,
 72,
 73,
 85,
 86,
 88,
 90,
 91,
 92,
 95,
 96,
 97,
 99,
 85,
 85]

In [13]:
node_ln_to_id = {}  #노드 라인넘버 -> id ex. 24(줄) -> 360087
for k,v in node_id_to_ln.items():   # node_id_to_ln은 extract_nodes 함수에서 추출되었음 k: id, v: ln
    if v in node_ln_to_id:
        continue
    node_ln_to_id[v] = k
node_id_to_function = {v:k for k,v in node_function_to_id.items()}


#adj리스트를 완성하고 이것을 combine_~ 함수를 호출하는 것 같은
adjacency_data = create_adjacency_data(node_ids,edges)

#함수 호출만을 KeyPoint로 설정
criterion_list = get_criterion("FC",node_ln_to_id,nodes_dict,False)

## 마지막에 주는 인자가 잘 이해가 안된다 ##
adjacency_list = create_adjacency_list_nodeID(node_ids, node_id_to_ln, edges, ['CONTROLS','REACHES'])
adjacency_list_REACHES = create_adjacency_list_nodeID(node_ids, node_id_to_ln, edges, ['REACHES','REACHES'])
adjacency_list_CONTROLS = create_adjacency_list_nodeID(node_ids, node_id_to_ln, edges, ['CONTROLS','CONTROLS'])
adjacency_list_IS_FUNCTION_OF_CFG = create_adjacency_list_nodeID(node_ids, node_id_to_ln, edges,['IS_FUNCTION_OF_CFG','IS_FUNCTION_OF_CFG'])

## 
combined_graph = combine_control_and_data_adjacents(adjacency_list)
combined_graph_REACHES = combine_control_and_data_adjacents(adjacency_list_REACHES)
combined_graph_CONTROLS = combine_control_and_data_adjacents(adjacency_list_CONTROLS)
combined_graph_FUNCTION_OF_CFG = combine_control_and_data_adjacents(adjacency_list_IS_FUNCTION_OF_CFG)


criterion_list

CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01_bad ()
good1 ()
CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01_good ()
main (int argc , char * argv [ ])


[['3600889', 'malloc'],
 ['3600939', 'memcpy'],
 ['3600996', 'free'],
 ['3601021', 'malloc'],
 ['3601071', 'memcpy'],
 ['3601128', 'free']]

In [12]:
slice_ln = criterion_list[0][0]
ln = slice_ln
forward_sliced_lines = create_backward_slice(combined_graph,ln )



##forward_sliced_lines
adjacency_list['3600985']

combined_graph
combined_graph_REACHES

forward_sliced_lines


['3600889', '3601006']

In [10]:
all_data_instance=[]

all_data_instance_last_index = -1

Sink_flag = False 
Source_flag = False

for function in node_function_to_id:
    if function.find("Sink") >= 0:
        Sink_flag = True
    elif function.find("Source") >= 0:
        Source_flag = True
            
_datas = set()

# criterion_list에 대한 각 데이터를 반복적으로 처리
# criterion_list에는 취약점 후보가 될 함수 들이 있다
for i, data in enumerate(criterion_list):
    data_instance = {}
    slice_ln = data[0]

## start 현재 라인에 대해서 모든 슬라이스 생성
    all_slice_ids = create_adjacency_slices(combined_graph, [slice_ln]) # foward, backward 합치기
    function_id = node_id_to_function_id[slice_ln]
    
    #해당 함수 레이블링
    label = calc_label(node_id_to_function, function_id, Sink_flag, Source_flag, False)
    
    #함수 진입접에 대한 제어 그래프의 해당 항목 삭제(?)
    try:
        del combined_graph_CONTROLS[function_id_to_entry[function_id]]
    except:
        pass
    
    # 인접 데이터 Id를 invert -> 왜?
    adjacency_data_to_id = invert_graph_data(combined_graph_REACHES)

## end

    all_slices = []
    all_slices_line = []
    slices_line = []

    target = set()
    datas = set()
    
## start 타겟 슬라이스 생성 및 함수 ID 추가
    key = ' '.join([str(i) for i in adjacency_data_to_id])
    if key not in _datas:
        # 제어 흐름 그래프에서의 타깃 슬라이스 생성
        target = create_adjacency_slices(combined_graph_CONTROLS,adjacency_data_to_id)
        # 해당 슬라이스 라인의 함수 ID를 타겟에 추가
        target.add(node_id_to_function_id[slice_ln])
        

## end

    # 타겟에 있는 각 인덱스에 대해 실제 코드라인으로 변환(코드 가젯 생성?)
    # 코드 슬라이스 생성(코드 가젯)
    all_slices_line = set()
    for idx in target:
        if idx in node_id_to_ln:
            all_slices_line.add(node_id_to_ln[idx])
    all_slices_line = sorted(list(all_slices_line))

    # 슬라이스된 코드 라인을 가공해 주석 제거
    tmp = []
    for idx,code_line in enumerate(all_slices_line):
        if code[code_line-1].find("/*") >0: # 여기 잘 안된다고 했었던걸로 기억
            tmp.append(code[code_line-1][:code[code_line-1].find("/*")]+'\n')
        else :
            tmp.append(code[code_line-1])

    # 결과 데이터를 인스턴스에 저장 
    data_instance['FileName'] = "CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01.c"
    data_instance['idx'] = all_data_instance_last_index
    data_instance['CWE-ID'] = "CWE-"+get_CWE("CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01.c")
    data_instance['criterion'] = data[1]
    data_instance['line'] = node_id_to_ln[data[0]]
    data_instance['label'] = label
    data_instance['slices'] = tmp

    # 모든 데이터 인스턴스 리스트에 현재 인스턴스 추가
    all_data_instance.append(data_instance)
    all_data_instance_last_index+=1 
    
    


In [11]:
all_data_instance

[{'FileName': 'CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01.c',
  'idx': -1,
  'CWE-ID': 'CWE-122',
  'criterion': 'malloc',
  'line': 36,
  'label': 1,
  'slices': ['void CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01_bad()\n',
   '        if (structCharVoid == NULL) {exit(-1);}\n',
   '        structCharVoid->voidSecond = (void *)SRC_STR;\n',
   '        printLine((char *)structCharVoid->voidSecond);\n',
   '        memcpy(structCharVoid->charFirst, SRC_STR, sizeof(*structCharVoid));\n',
   "        structCharVoid->charFirst[(sizeof(structCharVoid->charFirst)/sizeof(char))-1] = '\\0'; \n",
   '        printLine((char *)structCharVoid->charFirst);\n',
   '        printLine((char *)structCharVoid->voidSecond);\n',
   '        free(structCharVoid);\n',
   '        if (structCharVoid == NULL) {exit(-1);}\n',
   '        structCharVoid->voidSecond = (void *)SRC_STR;\n',
   '        printLine((char *)structCharVoid->voidSecond);\n',
   '        memcpy(stru

In [ ]:

# 메인함수 
# 이미 그래프가 완성되었다는 가정하에 진행하는 것으로 확인
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--criterion',help='["FC", "PU", "AU", "AE"]', default='FC')
    parser.add_argument('--csv', help='normalized csv files to process', default=r'data\cpg\CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01.c')
    parser.add_argument('--src', help='source c files to process', default=r'data\source')
    parser.add_argument('--output', default='../data/processed_data/FC/slice.json')
    parser.add_argument('--every_file_save', action='store_true')
    parser.add_argument('--genTest', action='store_true')

    args = parser.parse_args('')
    args = parser.parse_args([
        '--criterion', 'FC',
        '--csv', r'C:\Users\csw24\Projects\슬라이서\data\cpg\CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01.c',
        '--src', r'C:\Users\csw24\Projects\슬라이서\data\source\CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01.c',
        '--output', 'output.json',
        '--every_file_save'
    ])
    
    all_data_instance=[]

    all_data_instance_last_index = -1

    #파일 열기, 특정 디렉터리 내의 파일을 순회하는데, 파일 내의 디렉토리를 받는 줄 알아서 안돌아가는 것 같음
    for idx , filename in enumerate(tqdm(os.listdir(args.csv))):
        
        # join() 인자로 filename을 제거해서 잘 받아와짐
        nodes_file_path =os.path.join(args.csv, 'nodes.csv')
        edges_file_path =os.path.join(args.csv, 'edges.csv')
        print(nodes_file_path)
        print(edges_file_path)
        # nodes_file_path = r'C:\Users\csw24\Projects\슬라이서\data\cpg\CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01.c\nodes.csv'
        # edges_file_path = r'C:\Users\csw24\Projects\슬라이서\data\cpg\CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01.c\edges.csv'
        source_path = r'C:\Users\csw24\Projects\슬라이서\data\source\CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01.c'
        
        nc = open(nodes_file_path) 
        nodes_file = csv.DictReader(nc, delimiter='\t')
        nodes_dict = [node for node in nodes_file]
        
        # nodes.csv를 통해 정보 받기    
        # 받아오는 과정에서 os 이게 잘 안다뤄짐
        # nodes = read_csv(os.path.join(args.csv, filename, 'nodes.csv'))#######################################################################################
        # edges = read_csv(os.path.join(args.csv, filename, 'edges.csv'))
        
        nodes = read_csv(nodes_file_path)
        edges = read_csv(edges_file_path)        
        src  = source_path
        file1 = open(src,'r')
        code = file1.readlines()
        file1.close()
        
        
        # 노드 탐색 및 id, ln 등을 활용하기 위해 해당 함수를 통해 여러 정보를 수집
        function_id_to_callee, function_info, function_id_to_entry, node_id_to_function_id, node_function_to_id, node_indices, node_ids, line_numbers, node_id_to_ln = extract_nodes_with_location_info(nodes)
        node_ln_to_id = {}
        # k: id, v: ln
        for k,v in node_id_to_ln.items():
            if v in node_ln_to_id:
                continue
            node_ln_to_id[v] = k
        # 해당 id와 funcion을 dict로 생성하는데 이는 결국 function id 형태를 id: fucntion 형태로 바꾸는 것
        node_id_to_function = {v:k for k,v in node_function_to_id.items()}
        
        adjacency_data = create_adjacency_data(node_ids,edges)

        # 어차피, 함수 호출에 관련된 취약점 함수만들 수집하기 위함이므로, 아래 함수에 로직을 추가
        criterion_list = get_criterion(args.criterion, node_ln_to_id, nodes_dict, args.genTest)
        
        adjacency_list = create_adjacency_list_nodeID(node_ids, node_id_to_ln, edges, ['CONTROLS','REACHES'])
        adjacency_list_REACHES = create_adjacency_list_nodeID(node_ids, node_id_to_ln, edges, ['REACHES','REACHES'])
        adjacency_list_CONTROLS = create_adjacency_list_nodeID(node_ids, node_id_to_ln, edges, ['CONTROLS','CONTROLS'])
        adjacency_list_IS_FUNCTION_OF_CFG = create_adjacency_list_nodeID(node_ids, node_id_to_ln, edges,['IS_FUNCTION_OF_CFG','IS_FUNCTION_OF_CFG'])

        combined_graph = combine_control_and_data_adjacents(adjacency_list)
        combined_graph_REACHES = combine_control_and_data_adjacents(adjacency_list_REACHES)
        combined_graph_CONTROLS = combine_control_and_data_adjacents(adjacency_list_CONTROLS)
        combined_graph_FUNCTION_OF_CFG = combine_control_and_data_adjacents(adjacency_list_IS_FUNCTION_OF_CFG)


        Sink_flag = False 
        Source_flag = False
        if not args.genTest:
            for function in node_function_to_id:
                if function.find("Sink") >= 0:
                    Sink_flag = True
                elif function.find("Source") >= 0:
                    Source_flag = True
                    
        _datas = set()
        for i, data in enumerate(criterion_list):
            data_instance = {}
            slice_ln = data[0]

            # 각 Flow(CONTROL, REACHES)를 합집합으로 구한 그래프가 있으면 이에대한 forward, backward 슬라이스를 구해서 이를 결합
            all_slice_ids = create_adjacency_slices(combined_graph, [slice_ln])
            function_id = node_id_to_function_id[slice_ln]
            
            label = calc_label(node_id_to_function, function_id, Sink_flag, Source_flag, args.genTest)
            
            try:
                del combined_graph_CONTROLS[function_id_to_entry[function_id]]
            except:
                pass

            adjacency_data_to_id = invert_graph_data(combined_graph_REACHES)
            #combined_graph_CONTROLS = combine_control_and_data_adjacents(adjacency_list_CONTROLS)
            
            #ignore dependency
            # 여기서 왜  또 구하는지는 봐야함    
            all_slice_ids = create_adjacency_slices(combined_graph,all_slice_ids)

            
            all_slices = []
            all_slices_line = []
            slices_line = []

            target = set()
            datas = set()

#            for node_id in all_slice_ids:
#                datas = datas.union(adjacency_data[node_id])
            
#            if len(datas) == 0:
#                continue
                
#            key = ' '.join([str(i) for i in datas])
#            if key not in _datas:
#                _datas.add(key)
#            else :
#                all_data_instance[all_data_instance_last_index]['criterion'] = all_data_instance[all_data_instance_last_index]['criterion'] + ", " + str(data[1])
#                all_data_instance[all_data_instance_last_index]['line'] = str(all_data_instance[all_data_instance_last_index]['line']) + ", " + str(node_id_to_ln[slice_ln])
 
#                continue 
#            for tmp_data in datas:
#                for node_id in adjacency_data_to_id[tmp_data]:
#                    node_function_id = node_id_to_function_id[node_id]
#                    if  node_function_id== function_id:
#                        target.add(node_id)
            
#            target = create_adjacency_slices(combined_graph_CONTROLS,target)
#            target.add(node_id_to_function_id[slice_ln])
            key = ' '.join([str(i) for i in adjacency_data_to_id])
            if key not in _datas:
                target = create_adjacency_slices(combined_graph_CONTROLS,adjacency_data_to_id)
            target.add(node_id_to_function_id[slice_ln])
           
            all_slices_line = set()
            for idx in target:
                if idx in node_id_to_ln:
                    all_slices_line.add(node_id_to_ln[idx])
            all_slices_line = sorted(list(all_slices_line))
            
            tmp = []
            for idx,code_line in enumerate(all_slices_line):
                if code[code_line-1].find("/*") >0:
                    tmp.append(code[code_line-1][:code[code_line-1].find("/*")]+'\n')
                else :
                    tmp.append(code[code_line-1])

            data_instance['FileName'] = filename
            data_instance['idx'] = all_data_instance_last_index
            data_instance['CWE-ID'] = "CWE-"+get_CWE(filename)
            data_instance['criterion'] = data[1]
            data_instance['line'] = node_id_to_ln[data[0]]
            data_instance['label'] = label
            data_instance['slices'] = tmp
            
            
            all_data_instance.append(data_instance)
            all_data_instance_last_index+=1 
        if args.every_file_save :
            output_path = os.path.join( args.output, filename + ".json")
            with open(output_path, 'w') as fp:
                json.dump(all_data_instance, fp)
                fp.close()
            all_data_instance = []
            all_data_instance_last_index = -1
            
    if not args.every_file_save :        
        output_path = args.output

        with open(output_path, 'w') as fp:
            json.dump(all_data_instance, fp)
            fp.close()
            
    print("Done")

    pass

if __name__ =='__main__':
    main()


  0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\csw24\Projects\슬라이서\data\cpg\CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01.c\nodes.csv
C:\Users\csw24\Projects\슬라이서\data\cpg\CWE122_Heap_Based_Buffer_Overflow__char_type_overrun_memcpy_01.c\edges.csv


FileNotFoundError: [Errno 2] No such file or directory: 'output.json\\edges.csv.json'

In [5]:
data = ['StrNCat', 'getaddrinfo', '_ui64toa', 'fclose', 'pthread_mutex_lock', 'gets_s', 'sleep', 
           '_ui64tot', 'freopen_s', '_ui64tow', 'send', 'lstrcat', 'HMAC_Update', '__fxstat', 'StrCatBuff', 
           '_mbscat', '_mbstok_s', '_cprintf_s', 'ldap_search_init_page', 'memmove_s', 'ctime_s', 'vswprintf', 
           'vswprintf_s', '_snwprintf', '_gmtime_s', '_tccpy', '*RC6*', '_mbslwr_s', 'random', 
           '__wcstof_internal', '_wcslwr_s', '_ctime32_s', 'wcsncat*', 'MD5_Init', '_ultoa', 
           'snprintf', 'memset', 'syslog', '_vsnprintf_s', 'HeapAlloc', 'pthread_mutex_destroy', 
           'ChangeWindowMessageFilter', '_ultot', 'crypt_r', '_strupr_s_l', 'LoadLibraryExA', '_strerror_s', 
           'LoadLibraryExW', 'wvsprintf', 'MoveFileEx', '_strdate_s', 'SHA1', 'sprintfW', 'StrCatNW', 
           '_scanf_s_l', 'pthread_attr_init', '_wtmpnam_s', 'snscanf', '_sprintf_s_l', 'dlopen', 
           'sprintfA', 'timed_mutex', 'OemToCharA', 'ldap_delete_ext', 'sethostid', 'popen', 'OemToCharW', 
           '_gettws', 'vfork', '_wcsnset_s_l', 'sendmsg', '_mbsncat', 'wvnsprintfA', 'HeapFree', '_wcserror_s', 
           'realloc', '_snprintf*', 'wcstok', '_strncat*', 'StrNCpy', '_wasctime_s', 'push*', '_lfind_s', 
           'CC_SHA512', 'ldap_compare_ext_s', 'wcscat_s', 'strdup', '_chsize_s', 'sprintf_s', 'CC_MD4_Init', 
           'wcsncpy', '_wfreopen_s', '_wcsupr_s', '_searchenv_s', 'ldap_modify_ext_s', '_wsplitpath', 
           'CC_SHA384_Final', 'MD2', 'RtlCopyMemory', 'lstrcatW', 'MD4', 'MD5', '_wcstok_s_l', '_vsnwprintf_s', 
           'ldap_modify_s', 'strerror', '_lsearch_s', '_mbsnbcat_s', '_wsplitpath_s', 'MD4_Update', '_mbccpy_s', 
           '_strncpy_s_l', '_snprintf_s', 'CC_SHA512_Init', 'fwscanf_s', '_snwprintf_s', 'CC_SHA1', 'swprintf', 
           'fprintf', 'EVP_DigestInit_ex', 'strlen', 'SHA1_Init', 'strncat', '_getws_s', 'CC_MD4_Final', 
           'wnsprintfW', 'lcong48', 'lrand48', 'write', 'HMAC_Init', '_wfopen_s', 'wmemchr', '_tmakepath', 
           'wnsprintfA', 'lstrcpynW', 'scanf_s', '_mbsncpy_s_l', '_localtime64_s', 'fstream.open', '_wmakepath', 
           'Connection.open', '_tccat', 'valloc', 'setgroups', 'unlink', 'fstream.put', 'wsprintfA', '*SHA1*', 
           '_wsearchenv_s', 'ualstrcpyA', 'CC_MD5_Update', 'strerror_s', 'HeapCreate', 'ualstrcpyW', '__xstat', 
           '_wmktemp_s', 'StrCatChainW', 'ldap_search_st', '_mbstowcs_s_l', 'ldap_modify_ext', '_mbsset_s', 
           'strncpy_s', 'move', 'execle', 'StrCat', 'xrealloc', 'wcsncpy_s', '_tcsncpy*', 'execlp', 
           'RIPEMD160_Final', 'ldap_search_s', 'EnterCriticalSection', '_wctomb_s_l', 'fwrite', '_gmtime64_s', 
           'sscanf_s', 'wcscat', '_strupr_s', 'wcrtomb_s', 'VirtualLock', 'ldap_add_ext_s', '_mbscpy', 
           '_localtime32_s', 'lstrcpy', '_wcsncpy*', 'CC_SHA1_Init', '_getts', '_wfopen', '__xstat64', 
           'strcoll', '_fwscanf_s_l', '_mbslwr_s_l', 'RegOpenKey', 'makepath', 'seed48', 'CC_SHA256', 
           'sendto', 'execv', 'CalculateDigest', 'memchr', '_mbscpy_s', '_strtime_s', 'ldap_search_ext_s', 
           '_chmod', 'flock', '__fxstat64', '_vsntprintf', 'CC_SHA256_Init', '_itoa_s', '__wcserror_s', 
           '_gcvt_s', 'fstream.write', 'sprintf', 'recursive_mutex', 'strrchr', 'gethostbyaddr', '_wcsupr_s_l', 
           'strcspn', 'MD5_Final', 'asprintf', '_wcstombs_s_l', '_tcstok', 'free', 'MD2_Final', 'asctime_s', 
           '_alloca', '_wputenv_s', '_wcsset_s', '_wcslwr_s_l', 'SHA1_Update', 'filebuf.sputc', 'filebuf.sputn', 
           'SQLConnect', 'ldap_compare', 'mbstowcs_s', 'HMAC_Final', 'pthread_condattr_init', '_ultow_s', 'rand', 
           'ofstream.put', 'CC_SHA224_Final', 'lstrcpynA', 'bcopy', 'system', 'CreateFile*', 'wcscpy_s', 
           '_mbsnbcpy*', 'open', '_vsnwprintf', 'strncpy', 'getopt_long', 'CC_SHA512_Final', '_vsprintf_s_l', 
           'scanf', 'mkdir', '_localtime_s', '_snprintf', '_mbccpy_s_l', 'memcmp', 'final', '_ultoa_s', 
           'lstrcpyW', 'LoadModule', '_swprintf_s_l', 'MD5_Update', '_mbsnset_s_l', '_wstrtime_s', '_strnset_s', 
           'lstrcpyA', '_mbsnbcpy_s', 'mlock', 'IsBadHugeWritePtr', 'copy', '_mbsnbcpy_s_l', 'wnsprintf', 
           'wcscpy', 'ShellExecute', 'CC_MD4', '_ultow', '_vsnwprintf_s_l', 'lstrcpyn', 'CC_SHA1_Final', 
           'vsnprintf', '_mbsnbset_s', '_i64tow', 'SHA256_Init', 'wvnsprintf', 'RegCreateKey', 'strtok_s', 
           '_wctime32_s', '_i64toa', 'CC_MD5_Final', 'wmemcpy', 'WinExec', 'CreateDirectory*', 
           'CC_SHA256_Update', '_vsnprintf_s_l', 'jrand48', 'wsprintf', 'ldap_rename_ext_s', 'filebuf.open', 
           '_wsystem', 'SHA256_Update', '_cwscanf_s', 'wsprintfW', '_sntscanf', '_splitpath', 'fscanf_s', 
           'strpbrk', 'wcstombs_s', 'wscanf', '_mbsnbcat_s_l', 'strcpynA', 'pthread_cond_init', 'wcsrtombs_s', 
           '_wsopen_s', 'CharToOemBuffA', 'RIPEMD160_Update', '_tscanf', 'HMAC', 'StrCCpy', 'Connection.connect', 
           'lstrcatn', '_mbstok', '_mbsncpy', 'CC_SHA384_Update', 'create_directories', 'pthread_mutex_unlock', 
           'CFile.Open', 'connect', '_vswprintf_s_l', '_snscanf_s_l', 'fputc', '_wscanf_s', '_snprintf_s_l', 
           'strtok', '_strtok_s_l', 'lstrcatA', 'snwscanf', 'pthread_mutex_init', 'fputs', 'CC_SHA384_Init', 
           '_putenv_s', 'CharToOemBuffW', 'pthread_mutex_trylock', '__wcstoul_internal', '_memccpy', 
           '_snwprintf_s_l', '_strncpy*', 'wmemset', 'MD4_Init', '*RC4*', 'strcpyW', '_ecvt_s', 'memcpy_s', 
           'erand48', 'IsBadHugeReadPtr', 'strcpyA', 'HeapReAlloc', 'memcpy', 'ldap_rename_ext', 'fopen_s', 
           'srandom', '_cgetws_s', '_makepath', 'SHA256_Final', 'remove', '_mbsupr_s', 'pthread_mutexattr_init', 
           '__wcstold_internal', 'StrCpy', 'ldap_delete', 'wmemmove_s', '_mkdir', 'strcat', '_cscanf_s_l', 
           'StrCAdd', 'swprintf_s', '_strnset_s_l', 'close', 'ldap_delete_ext_s', 'ldap_modrdn', 'strchr', 
           '_gmtime32_s', '_ftcscat', 'lstrcatnA', '_tcsncat', 'OemToChar', 'mutex', 'CharToOem', 'strcpy_s', 
           'lstrcatnW', '_wscanf_s_l', '__lxstat64', 'memalign', 'MD2_Init', 'StrCatBuffW', 'StrCpyN', 'CC_MD5', 
           'StrCpyA', 'StrCatBuffA', 'StrCpyW', 'tmpnam_r', '_vsnprintf', 'strcatA', 'StrCpyNW', '_mbsnbset_s_l', 
           'EVP_DigestInit', '_stscanf', 'CC_MD2', '_tcscat', 'StrCpyNA', 'xmalloc', '_tcslen', '*MD4*', 
           'vasprintf', 'strxfrm', 'chmod', 'ldap_add_ext', 'alloca', '_snscanf_s', 'IsBadWritePtr', 'swscanf_s', 
           'wmemcpy_s', '_itoa', '_ui64toa_s', 'EVP_DigestUpdate', '__wcstol_internal', '_itow', 'StrNCatW', 
           'strncat_s', 'ualstrcpy', 'execvp', '_mbccat', 'EVP_MD_CTX_init', 'assert', 'ofstream.write', 
           'ldap_add', '_sscanf_s_l', 'drand48', 'CharToOemW', 'swscanf', '_itow_s', 'RIPEMD160_Init', 
           'CopyMemory', 'initstate', 'getpwuid', 'vsprintf', '_fcvt_s', 'CharToOemA', 'setuid', 'malloc', 
           'StrCatNA', 'strcat_s', 'srand', 'getwd', '_controlfp_s', 'olestrcpy', '__wcstod_internal', 
           '_mbsnbcat', 'lstrncat', 'des_*', 'CC_SHA224_Init', 'set*', 'vsprintf_s', 'SHA1_Final', '_umask_s', 
           'gets', 'setstate', 'wvsprintfW', 'LoadLibraryEx', 'ofstream.open', 'calloc', '_mbstrlen', 
           '_cgets_s', '_sopen_s', 'IsBadStringPtr', 'wcsncat_s', 'add*', 'nrand48', 'create_directory', 
           'ldap_search_ext', '_i64toa_s', '_ltoa_s', '_cwscanf_s_l', 'wmemcmp', '__lxstat', 'lstrlen', 
           'pthread_condattr_destroy', '_ftcscpy', 'wcstok_s', '__xmknod', 'pthread_attr_destroy', 'sethostname', 
           '_fscanf_s_l', 'StrCatN', 'RegEnumKey', '_tcsncpy', 'strcatW', 'AfxLoadLibrary', 'setenv', 'tmpnam', 
           '_mbsncat_s_l', '_wstrdate_s', '_wctime64_s', '_i64tow_s', 'CC_MD4_Update', 'ldap_add_s', '_umask', 
           'CC_SHA1_Update', '_wcsset_s_l', '_mbsupr_s_l', 'strstr', '_tsplitpath', 'memmove', '_tcscpy', 
           'vsnprintf_s', 'strcmp', 'wvnsprintfW', 'tmpfile', 'ldap_modify', '_mbsncat*', 'mrand48', 'sizeof', 
           'StrCatA', '_ltow_s', '*desencrypt*', 'StrCatW', '_mbccpy', 'CC_MD2_Init', 'RIPEMD160', 'ldap_search', 
           'CC_SHA224', 'mbsrtowcs_s', 'update', 'ldap_delete_s', 'getnameinfo', '*RC5*', '_wcsncat_s_l', 
           'DriverManager.getConnection', 'socket', '_cscanf_s', 'ldap_modrdn_s', '_wopen', 'CC_SHA256_Final', 
           '_snwprintf*', 'MD2_Update', 'strcpy', '_strncat_s_l', 'CC_MD5_Init', 'mbscpy', 'wmemmove', 
           'LoadLibraryW', '_mbslen', '*alloc', '_mbsncat_s', 'LoadLibraryA', 'fopen', 'StrLen', 'delete', 
           '_splitpath_s', 'CreateFileTransacted*', 'MD4_Final', '_open', 'CC_SHA384', 'wcslen', 'wcsncat', 
           '_mktemp_s', 'pthread_mutexattr_destroy', '_snwscanf_s', '_strset_s', '_wcsncpy_s_l', 'CC_MD2_Final', 
           '_mbstok_s_l', 'wctomb_s', 'MySQL_Driver.connect', '_snwscanf_s_l', '*_des_*', 'LoadLibrary', 
           '_swscanf_s_l', 'ldap_compare_s', 'ldap_compare_ext', '_strlwr_s', 'GetEnvironmentVariable', 
           'cuserid', '_mbscat_s', 'strspn', '_mbsncpy_s', 'ldap_modrdn2', 'LeaveCriticalSection', 'CopyFile', 
           'getpwd', 'sscanf', 'creat', 'RegSetValue', 'ldap_modrdn2_s', 'CFile.Close', '*SHA_1*', 
           'pthread_cond_destroy', 'CC_SHA512_Update', '*RC2*', 'StrNCatA', '_mbsnbcpy', '_mbsnset_s', 
           'crypt', 'excel', '_vstprintf', 'xstrdup', 'wvsprintfA', 'getopt', 'mkstemp', '_wcsnset_s', 
           '_stprintf', '_sntprintf', 'tmpfile_s', 'OpenDocumentFile', '_mbsset_s_l', '_strset_s_l', 
           '_strlwr_s_l', 'ifstream.open', 'xcalloc', 'StrNCpyA', '_wctime_s', 'CC_SHA224_Update', '_ctime64_s', 
           'MoveFile', 'chown', 'StrNCpyW', 'IsBadReadPtr', '_ui64tow_s', 'IsBadCodePtr', 'getc', 
           'OracleCommand.ExecuteOracleScalar', 'AccessDataSource.Insert', 'IDbDataAdapter.FillSchema', 
           'IDbDataAdapter.Update', 'GetWindowText*', 'SendMessage', 'SqlCommand.ExecuteNonQuery', 'streambuf.sgetc', 
           'streambuf.sgetn', 'OracleCommand.ExecuteScalar', 'SqlDataSource.Update', '_Read_s', 'IDataAdapter.Fill', 
           '_wgetenv', '_RecordsetPtr.Open*', 'AccessDataSource.Delete', 'Recordset.Open*', 'filebuf.sbumpc', 'DDX_*', 
           'RegGetValue', 'fstream.read*', 'SqlCeCommand.ExecuteResultSet', 'SqlCommand.ExecuteXmlReader', 'main', 
           'streambuf.sputbackc', 'read', 'm_lpCmdLine', 'CRichEditCtrl.Get*', 'istream.putback', 
           'SqlCeCommand.ExecuteXmlReader', 'SqlCeCommand.BeginExecuteXmlReader', 'filebuf.sgetn', 
           'OdbcDataAdapter.Update', 'filebuf.sgetc', 'SQLPutData', 'recvfrom', 'OleDbDataAdapter.FillSchema', 
           'IDataAdapter.FillSchema', 'CRichEditCtrl.GetLine', 'DbDataAdapter.Update', 'SqlCommand.ExecuteReader', 
           'istream.get', 'ReceiveFrom', '_main', 'fgetc', 'DbDataAdapter.FillSchema', 'kbhit', 'UpdateCommand.Execute*', 
           'Statement.execute', 'fgets', 'SelectCommand.Execute*', 'getch', 'OdbcCommand.ExecuteNonQuery', 
           'CDaoQueryDef.Execute', 'fstream.getline', 'ifstream.getline', 'SqlDataAdapter.FillSchema', 
           'OleDbCommand.ExecuteReader', 'Statement.execute*', 'SqlCeCommand.BeginExecuteNonQuery', 
           'OdbcCommand.ExecuteScalar', 'SqlCeDataAdapter.Update', 'sendmessage', 'mysqlpp.DBDriver', 'fstream.peek', 
           'Receive', 'CDaoRecordset.Open', 'OdbcDataAdapter.FillSchema', '_wgetenv_s', 'OleDbDataAdapter.Update', 
           'readsome', 'SqlCommand.BeginExecuteXmlReader', 'recv', 'ifstream.peek', '_Main', '_tmain', '_Readsome_s', 
           'SqlCeCommand.ExecuteReader', 'OleDbCommand.ExecuteNonQuery', 'fstream.get', 'IDbCommand.ExecuteScalar', 
           'filebuf.sputbackc', 'IDataAdapter.Update', 'streambuf.sbumpc', 'InsertCommand.Execute*', 'RegQueryValue', 
           'IDbCommand.ExecuteReader', 'SqlPipe.ExecuteAndSend', 'Connection.Execute*', 'getdlgtext', 'ReceiveFromEx', 
           'SqlDataAdapter.Update', 'RegQueryValueEx', 'SQLExecute', 'pread', 'SqlCommand.BeginExecuteReader', 'AfxWinMain', 
           'getchar', 'istream.getline', 'SqlCeDataAdapter.Fill', 'OleDbDataReader.ExecuteReader', 'SqlDataSource.Insert', 
           'istream.peek', 'SendMessageCallback', 'ifstream.read*', 'SqlDataSource.Select', 'SqlCommand.ExecuteScalar', 
           'SqlDataAdapter.Fill', 'SqlCommand.BeginExecuteNonQuery', 'getche', 'SqlCeCommand.BeginExecuteReader', 'getenv', 
           'streambuf.snextc', 'Command.Execute*', '_CommandPtr.Execute*', 'SendNotifyMessage', 'OdbcDataAdapter.Fill', 
           'AccessDataSource.Update', 'fscanf', 'QSqlQuery.execBatch', 'DbDataAdapter.Fill', 'cin', 
           'DeleteCommand.Execute*', 'QSqlQuery.exec', 'PostMessage', 'ifstream.get', 'filebuf.snextc', 
           'IDbCommand.ExecuteNonQuery', 'Winmain', 'fread', 'getpass', 'GetDlgItemTextCCheckListBox.GetCheck', 
           'DISP_PROPERTY_EX', 'pread64', 'Socket.Receive*', 'SACommand.Execute*', 'SQLExecDirect', 
           'SqlCeDataAdapter.FillSchema', 'DISP_FUNCTION', 'OracleCommand.ExecuteNonQuery', 'CEdit.GetLine', 
           'OdbcCommand.ExecuteReader', 'CEdit.Get*', 'AccessDataSource.Select', 'OracleCommand.ExecuteReader', 
           'OCIStmtExecute', 'getenv_s', 'DB2Command.Execute*', 'OracleDataAdapter.FillSchema', 'OracleDataAdapter.Fill', 
           'CComboBox.Get*', 'SqlCeCommand.ExecuteNonQuery', 'OracleCommand.ExecuteOracleNonQuery', 'mysqlpp.Query', 
           'istream.read*', 'CListBox.GetText', 'SqlCeCommand.ExecuteScalar', 'ifstream.putback', 'readlink', 
           'CHtmlEditCtrl.GetDHtmlDocument', 'PostThreadMessage', 'CListCtrl.GetItemText', 'OracleDataAdapter.Update', 
           'OleDbCommand.ExecuteScalar', 'stdin', 'SqlDataSource.Delete', 'OleDbDataAdapter.Fill', 'fstream.putback', 
           'IDbDataAdapter.Fill', '_wspawnl', 'fwprintf', 'sem_wait', '_unlink', 'ldap_search_ext_sW', 'signal', 'PQclear', 
           'PQfinish', 'PQexec', 'PQresultStatus', 'atoi']
file_name = "sensiAPI.txt"
with open(file_name, "w", encoding="utf-8") as file:
    # 각 문자열을 쉼표와 공백으로 연결하여 파일에 쓰기
    file.write(", ".join(data))


In [22]:
file_path = r"C:\Users\user\Desktop\slices_AU.json"
with open(file_path, 'r') as file:
    data = json.load(file)

unique_filename_list = []
data_dict = {}

for instnace in data:
    CWE = instnace["CWE-ID"] 
    label = instnace["label"] 
    FileName = instnace['FileName']
    #criterion = instnace["criterion"] 
    Filtered_FileName = "_".join(FileName.split('.')[0].split('_')[:-1])
    if Filtered_FileName not in unique_filename_list:
        if CWE not in data_dict:
            data_dict[CWE] = {}
        if label not in data_dict[CWE]:
            data_dict[CWE][label] = []
        data_dict[CWE][label].append(instnace)
        unique_filename_list.append(Filtered_FileName)


In [44]:
root_path = r"C:\Users\user\Desktop\학습데이터 유형별 정리(AU, unique)"
for key in data_dict:
    file_name = key

    if 0 in data_dict[key]:
        non_vul = data_dict[key][0]
        nonvul_path = os.path.join(root_path, file_name + "_Non-vul.json")
    
        with open(nonvul_path, "w") as json_file:
            json.dump(non_vul, json_file)
    
    if 1 in data_dict[key]:
    
        vul = data_dict[key][1]
        vul_path = os.path.join(root_path, file_name + "_vul.json")
    
        with open(vul_path, "w") as json_file:
            json.dump(vul, json_file)

NameError: name 'data_dict' is not defined